#### Instala as dependências da aplicação
###### Baseado em: https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/

In [1]:
import sys
!{sys.executable} -m pip install pandas requests

#### Importa das bibliotecas necessárias

In [2]:
import requests
import pandas as pd

#### Define os parâmetros globais do programa

In [3]:
BASE_URL = ""
LOGIN_URL = "{}/oauth/token".format(BASE_URL)
OBJECTIVES_LIST_URL = "{}/api/objectives".format(BASE_URL)
CLIENT_ID = ""
CLIENT_SECRET = ""
GRANT_TYPE = ""
EMAIL = ""
PASSWORD = ""

#### Cria uma factory de requisições HTTP

In [4]:
# Factory de requisições HTTP
def do_request(verb: str, url: str, token: str = None, data: dict = {}, params: dict = {}) -> str:
    headers = {'Authorization': token}

    method = getattr(requests, verb.lower())
    request = method(url, headers=headers, params=params, data=data)
    result = request.json()

    if 'data' in result:
        result = result['data']

    return result

#### Função responsável por obter o token de acesso à API da Coblue

In [7]:
def auth() -> str:
    payload = {
        "username": EMAIL,
        "password": PASSWORD,
        "client_id": CLIENT_ID,
        "grant_type": GRANT_TYPE,
        "client_secret": CLIENT_SECRET
    }

    response = do_request('POST', LOGIN_URL, data=payload)

    token_type = response['token_type']
    access_token = response['access_token']

    return "{} {}".format(token_type, access_token)

#### Função responsável por obter todos os OKR's da conta

In [8]:
def get_okrs(token: str) -> list:
    params = {
        'with': 'user;keyResults;section;keyResults.user',
    }

    return do_request("GET", OBJECTIVES_LIST_URL, token, params=params)

#### Funções responsáveis por salvar os dados dos OKR's para um arquivo Excel

In [9]:
def save_okrs_to_spreadsheet_file(data: list) -> None:
    df_objectives = generate_objectives_dataframe(data)
    df_key_results = generate_key_results_dataframe(data)

    writer = pd.ExcelWriter('dados_okr.xlsx')
    df_objectives.to_excel(writer, sheet_name='objectives')
    df_key_results.to_excel(writer, sheet_name='key_results')

    writer.save()


def generate_objectives_dataframe(data: list) -> pd.DataFrame:
    columns = [
        'id', 'user_id', 'parent_id', 'section_id', 
        'period_id', 'progress', 'created_at'
    ]
    return pd.DataFrame(data, columns=columns)


def generate_key_results_dataframe(data: list) -> pd.DataFrame:
    key_results = []

    for objective in data:
        key_results.extend(objective['key_results'])

    columns = ['id', 'objective_id', 'user_id', 'progress', 'created_at']
    return pd.DataFrame(key_results, columns=columns)

#### Execução do processo de extração dos dados

In [10]:
# Gera o token de acesso a API da plataforma
token = auth()

# Requisita todos os OKR's em conjunto com os seus Resultados Chaves,
# usuário responsável, objetivo pai e unidade ou departamento ao qual
# o objetivo está atrelado
okrs = get_okrs(token)

# Salva a resposta da API que é obtida no formato JSON para
# um arquivo de planilha contendo os dados para uso posterior
save_okrs_to_spreadsheet_file(okrs)